### **To work with TensorFlow-CPU**
import os</br>
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'</br>
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'</br>

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

**Dataset**

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

* Cement
* Blast furnace slag
* Fly ash
* Water
* Superplasticizer
* Coarse aggregate
* Fine aggregate

**Loading Datasets**

In [2]:
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
df = pd.read_csv(url)
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

**Lets check the shpae of the data**

In [3]:
df.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


**Lets check for any missing value**

In [4]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [5]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


**Lets split the dataset into predictors and target**

Predictors: All columns other than the target column i.e. except predicting column</br>
Target: Column to be predicted

In [6]:
concrete_data_columns = df.columns
predictors = df[concrete_data_columns[concrete_data_columns!='Strength']]
target = df.Strength

We have our predictor and target DataFrame.

In [7]:
print(predictors.head())
print(target.head())

   Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  
0            1040.0           676.0   28  
1            1055.0           676.0   28  
2             932.0           594.0  270  
3             932.0           594.0  365  
4             978.4           825.5  360  
0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64


Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [8]:
predictors_norm = (predictors-predictors.mean())/predictors.std()
predictors_norm.sample(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
288,-0.954658,-0.856472,1.762765,-0.560418,0.233575,1.063368,0.052628,0.860186
1024,-1.102018,2.153519,-0.846733,0.076459,1.087297,-1.467693,0.663784,-0.279597
80,1.376300,0.375573,-0.846733,-1.314367,1.723404,-1.553862,1.415879,-0.675355
320,-0.306851,-0.856472,0.697090,-1.098953,1.104037,0.191387,1.439577,-0.501222
538,1.902584,-0.856472,-0.846733,0.488555,-1.038638,-0.472245,-0.765572,-0.612034


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [9]:
n_cols = predictors_norm.shape[1]

**Lets import important `Kera` packages**

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

**Building a Model**

Lets define a function that can be used for building model.

In [11]:
def regression_model():
    # Build model
    model = Sequential()
    model.add(Input(shape = (n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='mean_squared_error')
    return model

**Train and Test the Model**

In [12]:
model = regression_model()

2025-05-03 16:37:05.832994: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-05-03 16:37:05.833020: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-03 16:37:05.833028: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-03 16:37:05.833056: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-03 16:37:05.833071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [16]:
model.fit(x=predictors_norm, y=target, validation_split=0.3, epochs=100, verbose=True)

Epoch 1/100
23/23 [==============================] - 0s 5ms/step - loss: 1658.0022 - val_loss: 1174.7974
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 1547.9407 - val_loss: 1099.7601
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 1440.9368 - val_loss: 1021.1656
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 1316.7897 - val_loss: 934.8947
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 1176.6882 - val_loss: 841.4799
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 1033.4349 - val_loss: 750.2767
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 901.8964 - val_loss: 655.7469
Epoch 8/100
23/23 [==============================] - 0s 5ms/step - loss: 782.5415 - val_loss: 571.2643
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 681.0488 - val_loss: 489.1803
Epoch 10/100
23/23 [==============================] - 0s 7ms/ste

**Lets try with 5 number of hidden layers with 50 node each**

In [17]:
def model_building():
    model = Sequential()
    model.add(Input(shape=(n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
model_ = model_building()
model_.fit(predictors_norm, target, validation_split=0.3, verbose=2, epochs=100)

Epoch 1/100
23/23 - 0s - loss: 1665.0046 - val_loss: 1198.5740 - 435ms/epoch - 19ms/step
Epoch 2/100
23/23 - 0s - loss: 1528.0211 - val_loss: 1193.9980 - 113ms/epoch - 5ms/step
Epoch 3/100
23/23 - 0s - loss: 1445.1848 - val_loss: 1233.3304 - 150ms/epoch - 7ms/step
Epoch 4/100
23/23 - 0s - loss: 1348.8033 - val_loss: 1062.0958 - 112ms/epoch - 5ms/step
Epoch 5/100
23/23 - 0s - loss: 1245.7452 - val_loss: 1355.5321 - 116ms/epoch - 5ms/step
Epoch 6/100
23/23 - 0s - loss: 4292.9307 - val_loss: 2630.7761 - 121ms/epoch - 5ms/step
Epoch 7/100
23/23 - 0s - loss: 3778.0906 - val_loss: 643.3236 - 113ms/epoch - 5ms/step
Epoch 8/100
23/23 - 0s - loss: 4011.8027 - val_loss: 584.6342 - 112ms/epoch - 5ms/step
Epoch 9/100
23/23 - 0s - loss: 2556.9578 - val_loss: 7002.3237 - 112ms/epoch - 5ms/step
Epoch 10/100
23/23 - 0s - loss: 26367.2480 - val_loss: 7829.0913 - 111ms/epoch - 5ms/step
Epoch 11/100
23/23 - 0s - loss: 7623.5298 - val_loss: 726.2114 - 111ms/epoch - 5ms/step
Epoch 12/100
23/23 - 0s - loss:

Based on the results, we notice that:

- Adding more hidden layers to the model increases its capacity to learn and represent complex relationships within the data. This allows the model to better identify, as a result, the model becomes more effective at fitting the training data and potentially improving its predictions.
- By reducing the proportion of data set aside for validation and using a larger portion for training, the model has access to more examples to learn from. This additional training data helps the model improve its understanding of the underlying trends, which can lead to better overall performance.  


**Make Prediction**

In [23]:
X, y = predictors_norm, target
y_pred = model.predict(X)
for actual, predicted in zip(y, y_pred):
    print(f'Actual: {actual:.3f}, Predicted: {predicted[0]:.3f}')

33/33 [==============================] - 0s 1ms/step
Actual: 79.990, Predicted: 53.276
Actual: 61.890, Predicted: 53.089
Actual: 40.270, Predicted: 58.765
Actual: 41.050, Predicted: 70.203
Actual: 44.300, Predicted: 61.736
Actual: 47.030, Predicted: 27.971
Actual: 43.700, Predicted: 70.851
Actual: 36.450, Predicted: 30.274
Actual: 45.850, Predicted: 20.505
Actual: 39.290, Predicted: 31.569
Actual: 38.070, Predicted: 29.227
Actual: 28.020, Predicted: 21.762
Actual: 43.010, Predicted: 60.060
Actual: 42.330, Predicted: 26.935
Actual: 47.810, Predicted: 21.023
Actual: 52.910, Predicted: 29.524
Actual: 39.360, Predicted: 29.272
Actual: 56.140, Predicted: 62.118
Actual: 40.560, Predicted: 37.739
Actual: 42.620, Predicted: 49.871
Actual: 41.840, Predicted: 49.223
Actual: 28.240, Predicted: 21.807
Actual: 8.060, Predicted: 18.797
Actual: 44.210, Predicted: 40.108
Actual: 52.520, Predicted: 62.636
Actual: 53.300, Predicted: 51.197
Actual: 41.150, Predicted: 59.412
Actual: 52.120, Predicted: 39.